In [3]:
#df.to_csv('cmb_clean_data.csv',index=False)

import pandas as pd
import numpy as np

df=pd.read_csv('cmb_clean_data.csv')

y= df['Price']
y = y[:5872]
y = pd.DataFrame(y)                   
df=df.drop('Price', 1)
y.isna().sum()

Price    0
dtype: int64

In [4]:
df.isna().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                31
Seats                 0
dtype: int64

In [5]:
df['Power'] = df['Power'].fillna((df['Power'].mean()))
df['Engine'] = df['Engine'].fillna((df['Engine'].mean()))

In [6]:
X=df
X.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,17,9,12,72000,0,1,0,26.60,998.0,58.16,2
1,10,10,17,41000,1,1,0,19.67,1582.0,126.20,2
2,9,2,13,46000,3,1,0,18.20,1199.0,88.70,2
3,17,2,14,87000,1,1,0,20.77,1248.0,88.76,4
4,1,3,15,40670,1,0,2,15.20,1968.0,140.80,2


In [7]:


from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score  

import warnings
warnings.filterwarnings('ignore')

enc = OneHotEncoder(handle_unknown='ignore')


onehotencoder = OneHotEncoder(categorical_features=[0])
X = onehotencoder.fit_transform(X).toarray()
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()
onehotencoder = OneHotEncoder(categorical_features=[2])
X = onehotencoder.fit_transform(X).toarray()
onehotencoder = OneHotEncoder(categorical_features=[4])
X = onehotencoder.fit_transform(X).toarray()
onehotencoder = OneHotEncoder(categorical_features=[5])
X = onehotencoder.fit_transform(X).toarray()
onehotencoder = OneHotEncoder(categorical_features=[10])
X = onehotencoder.fit_transform(X).toarray()
X = pd.DataFrame(X)
X.tail()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
7100,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,5.0,13.0,89411.0,1.0,1.0,0.0,20.54,1598.0,103.6,2.0
7101,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,9.0,17.0,59000.0,3.0,0.0,0.0,17.21,1197.0,103.6,2.0
7102,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,8.0,14.0,28000.0,1.0,1.0,0.0,23.08,1461.0,63.1,2.0
7103,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,10.0,15.0,52262.0,3.0,0.0,3.0,17.20,1197.0,103.6,2.0
7104,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,7.0,16.0,72443.0,1.0,0.0,0.0,10.00,2148.0,170.0,2.0


In [8]:
from sklearn import preprocessing
scaler_x = preprocessing.MinMaxScaler ( feature_range =( -1, 1))
x = np. array (X).reshape ((len( X) ,45))
x = scaler_x.fit_transform (x)

scaler_y = preprocessing. MinMaxScaler ( feature_range =( -1, 1))
y = np.array (y).reshape ((len( y), 1))
y = scaler_y.fit_transform (y)


In [9]:
train_df = x[:5872]
test_df = x[-1233:]
print(train_df.shape[0])
print(test_df.shape[0])

5872
1233


In [10]:
from sklearn.ensemble import  GradientBoostingRegressor

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, 
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)


GBoost.fit(train_df,y)

#preds = GBoost.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=4,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=5,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [21]:
pred = GBoost.predict(test_df)
pred

array([-0.96741726, -0.96642542, -0.79170228, ..., -0.97045333,
       -0.93964166, -0.80614383])

In [22]:
pred = scaler_y.inverse_transform (np. array (pred). reshape ((-1, 1)))

pred

array([[ 3.03945089],
       [ 3.11857976],
       [17.0579923 ],
       ...,
       [ 2.79723314],
       [ 5.25538858],
       [15.90584556]])

In [23]:
predictions = pd.DataFrame(pred)
predictions.head()

,0
0,3.039451
1,3.118580
2,17.057992
3,5.755657
4,4.578309


In [24]:
predictions.to_csv('pred_data.csv',index=False)